In [1]:
# module imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import random
from sklearn.decomposition import PCA

# model imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import KernelPCA


# processing imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef, precision_score, recall_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score,classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(1635848)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# fetch the training file
file_path_full_training_set = '/content/drive/MyDrive/KDD-NSL/KDDTrain+.txt'
file_path_test = '/content/drive/MyDrive/KDD-NSL/KDDTest+.txt'

In [4]:
df = pd.read_csv(file_path_full_training_set)
test_df = pd.read_csv(file_path_test)

In [5]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

In [6]:
df.columns = columns
test_df.columns = columns

In [7]:
# Categorical features analysis
print('Training set categorical features:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Training set categorical features:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'attack' has 23 categories


In [8]:
# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Training set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'attack' has 23 categories


In [9]:
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())

Distribution of categories in service:
service
http        40338
private     21853
domain_u     9043
smtp         7313
ftp_data     6859
Name: count, dtype: int64


In [10]:
# Test set
print('Test set:')
for col_name in test_df.columns:
    if test_df[col_name].dtypes == 'object' :
        unique_cat = len(test_df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Test set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 64 categories
Feature 'flag' has 11 categories
Feature 'attack' has 38 categories


In [11]:
# Attack Distribution Comparison Analysis
print("="*60)
print("ATTACK TYPE DISTRIBUTION ANALYSIS")
print("="*60)

print("\nTraining Set Attack Distribution:")
print(df['attack'].value_counts().head(10))

print("\nTest Set Attack Distribution:")
print(test_df['attack'].value_counts().head(10))

print("\nUnique Attacks Comparison:")
train_attacks = set(df['attack'].unique())
test_attacks = set(test_df['attack'].unique())

print(f"Training attacks: {len(train_attacks)} unique types")
print(f"Test attacks: {len(test_attacks)} unique types")

attacks_only_in_train = train_attacks - test_attacks
attacks_only_in_test = test_attacks - train_attacks
common_attacks = train_attacks.intersection(test_attacks)

print(f"\nAttacks ONLY in training: {len(attacks_only_in_train)}")
if attacks_only_in_train:
    print(f"   {list(attacks_only_in_train)}")

print(f"\nAttacks ONLY in test: {len(attacks_only_in_test)}")
if attacks_only_in_test:
    print(f"   {list(attacks_only_in_test)}")

print(f"\nCommon attacks: {len(common_attacks)}")

ATTACK TYPE DISTRIBUTION ANALYSIS

Training Set Attack Distribution:
attack
normal         67342
neptune        41214
satan           3633
ipsweep         3599
portsweep       2931
smurf           2646
nmap            1493
back             956
teardrop         892
warezclient      890
Name: count, dtype: int64

Test Set Attack Distribution:
attack
normal          9711
neptune         4656
guess_passwd    1231
mscan            996
warezmaster      944
apache2          737
satan            735
processtable     685
smurf            665
back             359
Name: count, dtype: int64

Unique Attacks Comparison:
Training attacks: 23 unique types
Test attacks: 38 unique types

Attacks ONLY in training: 2
   ['spy', 'warezclient']

Attacks ONLY in test: 17
   ['udpstorm', 'named', 'httptunnel', 'processtable', 'apache2', 'mailbomb', 'mscan', 'sendmail', 'xlock', 'snmpguess', 'snmpgetattack', 'xterm', 'ps', 'xsnoop', 'worm', 'sqlattack', 'saint']

Common attacks: 21


In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define categorical columns for encoding
categorical_columns = ['protocol_type', 'service', 'flag']

# Extract categorical values
df_categorical_values = df[categorical_columns]
testdf_categorical_values = test_df[categorical_columns]
df_categorical_values.head()

,protocol_type,service,flag
0,udp,other,SF
1,tcp,private,S0
2,tcp,http,SF
3,tcp,http,SF
4,tcp,private,REJ


In [13]:
# Generate column names for categorical encoding
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]

# Use combined unique services from both train and test sets
unique_service_train = set(df.service.unique())
unique_service_test = set(test_df.service.unique())
unique_service_combined = sorted(list(unique_service_train.union(unique_service_test)))
string2 = 'service_'
unique_service2 = [string2 + x for x in unique_service_combined]

# Flag columns
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]

# Combined columns for consistent encoding
dumcols = unique_protocol2 + unique_service2 + unique_flag2
testdumcols = dumcols

print(f"Total columns: {len(dumcols)}")
print(f"Protocol columns: {len(unique_protocol2)}")
print(f"Service columns: {len(unique_service2)}")
print(f"Flag columns: {len(unique_flag2)}")

print(f"\nServices only in train: {unique_service_train - unique_service_test}")
print(f"Services only in test: {unique_service_test - unique_service_train}")
print(f"Combined services: {len(unique_service_combined)}")

Total columns: 84
Protocol columns: 3
Service columns: 70
Flag columns: 11

Services only in train: {'harvest', 'red_i', 'aol', 'urh_i', 'http_2784', 'http_8001'}
Services only in test: set()
Combined services: 70


In [14]:
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

   protocol_type  service  flag
0              2       44     9
1              1       49     5
2              1       24     9
3              1       24     9
4              1       49     1


In [15]:
# OneHot encoding with proper handling of different categorical values
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Combine train and test categorical data to capture all possible categories
combined_categorical = pd.concat([df_categorical_values_enc, testdf_categorical_values_enc], ignore_index=True)

# Fit encoder on combined data to handle all categories consistently
enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
enc.fit(combined_categorical)

# Transform train and test sets
df_categorical_values_encenc = enc.transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc, columns=dumcols)

testdf_categorical_values_encenc = enc.transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc, columns=dumcols)

print(f"Train set categorical shape: {df_cat_data.shape}")
print(f"Test set categorical shape: {testdf_cat_data.shape}")
print(f"Column names match: {list(df_cat_data.columns) == list(testdf_cat_data.columns)}")

df_cat_data.head()

Train set categorical shape: (125972, 84)
Test set categorical shape: (22543, 84)
Column names match: True


,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Verify column structure consistency
print("Column structure verification:")
print(f"Train categorical shape: {df_cat_data.shape}")
print(f"Test categorical shape: {testdf_cat_data.shape}")
print(f"Columns match: {df_cat_data.shape[1] == testdf_cat_data.shape[1]}")

Column structure verification:
Train categorical shape: (125972, 84)
Test categorical shape: (22543, 84)
Columns match: True


In [17]:
# Final verification of column structure consistency
print("Column structure consistency achieved")

print(f"Final train categorical shape: {df_cat_data.shape}")
print(f"Final test categorical shape: {testdf_cat_data.shape}")
assert df_cat_data.shape[1] == testdf_cat_data.shape[1], "Column count mismatch!"
print("Verification complete - ready to proceed")

Column structure consistency achieved
Final train categorical shape: (125972, 84)
Final test categorical shape: (22543, 84)
Verification complete - ready to proceed


In [18]:
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=test_df.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=test_df.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)

(125972, 124)
(22543, 124)
(125972, 124)
(22543, 124)


In [19]:
# take label column
labeldf=newdf['attack']
labeldf_test=newdf_test['attack']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['attack'] = newlabeldf
newdf_test['attack'] = newlabeldf_test
print(newdf['attack'].head())


0    0
1    1
2    0
3    0
4    1
Name: attack, dtype: int64


/tmp/ipython-input-1623462165.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
/tmp/ipython-input-1623462165.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstor

In [20]:
to_drop_DoS = [2,3,4]
to_drop_Probe = [1,3,4]
to_drop_R2L = [1,2,4]
to_drop_U2R = [1,2,3]
DoS_df=newdf[~newdf['attack'].isin(to_drop_DoS)];
Probe_df=newdf[~newdf['attack'].isin(to_drop_Probe)];
R2L_df=newdf[~newdf['attack'].isin(to_drop_R2L)];
U2R_df=newdf[~newdf['attack'].isin(to_drop_U2R)];

#test
DoS_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_DoS)];
Probe_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_Probe)];
R2L_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_R2L)];
U2R_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_U2R)];
print('Train:')
print('Dimensions of DoS:' ,DoS_df.shape)
print('Dimensions of Probe:' ,Probe_df.shape)
print('Dimensions of R2L:' ,R2L_df.shape)
print('Dimensions of U2R:' ,U2R_df.shape)
print('Test:')
print('Dimensions of DoS:' ,DoS_df_test.shape)
print('Dimensions of Probe:' ,Probe_df_test.shape)
print('Dimensions of R2L:' ,R2L_df_test.shape)
print('Dimensions of U2R:' ,U2R_df_test.shape)

Train:
Dimensions of DoS: (113269, 124)
Dimensions of Probe: (78998, 124)
Dimensions of R2L: (68337, 124)
Dimensions of U2R: (67394, 124)
Test:
Dimensions of DoS: (17170, 124)
Dimensions of Probe: (12132, 124)
Dimensions of R2L: (12596, 124)
Dimensions of U2R: (9778, 124)


In [21]:
# Split dataframes into X & Y
# assign X as a dataframe of feautures and Y as a series of outcome variables
X_DoS = DoS_df.drop('attack', axis=1)
Y_DoS = DoS_df.attack
X_Probe = Probe_df.drop('attack', axis=1)
Y_Probe = Probe_df.attack
X_R2L = R2L_df.drop('attack', axis=1)
Y_R2L = R2L_df.attack
X_U2R = U2R_df.drop('attack', axis=1)
Y_U2R = U2R_df.attack
# test set
X_DoS_test = DoS_df_test.drop('attack', axis=1)
Y_DoS_test = DoS_df_test.attack
X_Probe_test = Probe_df_test.drop('attack', axis=1)
Y_Probe_test = Probe_df_test.attack
X_R2L_test = R2L_df_test.drop('attack', axis=1)
Y_R2L_test = R2L_df_test.attack
X_U2R_test = U2R_df_test.drop('attack', axis=1)
Y_U2R_test = U2R_df_test.attack

In [22]:
colNames=list(X_U2R)
colNames_test=list(X_U2R_test)

# Analyze extreme U2R class imbalance
print("U2R Dataset Class Distribution Analysis:")
print(f"Training set:")
print(f"  Normal (0): {np.sum(Y_U2R == 0):,} samples ({np.sum(Y_U2R == 0)/len(Y_U2R)*100:.3f}%)")
print(f"  U2R (4): {np.sum(Y_U2R == 4):,} samples ({np.sum(Y_U2R == 4)/len(Y_U2R)*100:.3f}%)")
print(f"  Imbalance ratio: {np.sum(Y_U2R == 0) / max(np.sum(Y_U2R == 4), 1):.1f}:1")
print(f"Challenge: EXTREME class imbalance - U2R represents only {np.sum(Y_U2R == 4)/len(Y_U2R)*100:.4f}% of data!")

U2R Dataset Class Distribution Analysis:
Training set:
  Normal (0): 67,342 samples (99.923%)
  U2R (4): 52 samples (0.077%)
  Imbalance ratio: 1295.0:1
Challenge: EXTREME class imbalance - U2R represents only 0.0772% of data!


In [23]:
from sklearn import preprocessing

# Feature scaling - fit on U2R training data, transform both train and test
scaler_U2R = preprocessing.StandardScaler().fit(X_U2R)
X_U2R = scaler_U2R.transform(X_U2R)
X_U2R_test = scaler_U2R.transform(X_U2R_test)

# Convert U2R labels to binary (0: Normal, 1: U2R Attack)
Y_U2R_binary = np.where(Y_U2R == 4, 1, 0)
Y_U2R_test_binary = np.where(Y_U2R_test == 4, 1, 0)

print(f"After binary conversion:")
print(f"  Training - Normal: {np.sum(Y_U2R_binary == 0):,}, U2R: {np.sum(Y_U2R_binary == 1):,}")
print(f"  Test - Normal: {np.sum(Y_U2R_test_binary == 0):,}, U2R: {np.sum(Y_U2R_test_binary == 1):,}")

After binary conversion:
  Training - Normal: 67,342, U2R: 52
  Test - Normal: 9,711, U2R: 67


In [24]:
# Verify standardization results for U2R data
print("Standard deviation check (should be ~1.0):")
print(X_U2R.std(axis=0)[:10])  # Show first 10 features

Standard deviation check (should be ~1.0):
[1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]


In [25]:
from sklearn.feature_selection import SelectPercentile, f_classif
np.seterr(divide='ignore', invalid='ignore');
selector=SelectPercentile(f_classif, percentile=15)  # Increased for U2R
X_newU2R = selector.fit_transform(X_U2R, Y_U2R_binary)
X_newU2R.shape

/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [  4  16  44  45  47  48  49  50  51  52  55  58  59  63  64  65  67  68
  69  71  72  73  74  75  76  77  78  79  80  81  82  83  84  87  88  90
  93  94  97  99 100 101 108 109 110 111 115] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


(67394, 19)

In [26]:
true=selector.get_support()
newcolindex_U2R=[i for i, x in enumerate(true) if x]
newcolname_U2R=list( colNames[i] for i in newcolindex_U2R )
newcolname_U2R

['urgent',
 'hot',
 'logged_in',
 'root_shell',
 'num_file_creations',
 'num_shells',
 'srv_count',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'level',
 'Protocol_type_tcp',
 'service_domain_u',
 'service_ftp',
 'service_ftp_data',
 'service_http',
 'service_telnet']

In [27]:
from sklearn.feature_selection import RFE
clf = DecisionTreeClassifier(random_state=0)
rfe = RFE(estimator=clf, n_features_to_select=20, step=1)  # More features for U2R
rfe.fit(X_U2R, Y_U2R_binary)
X_rfeU2R=rfe.transform(X_U2R)
true=rfe.support_
rfecolindex_U2R=[i for i, x in enumerate(true) if x]
rfecolname_U2R=list(colNames[i] for i in rfecolindex_U2R)

In [28]:
# Display selected features for U2R classification
print('Features selected for U2R classification:')
for feature in rfecolname_U2R:
    print(f"  - {feature}")

Features selected for U2R classification:
  - dst_bytes
  - srv_count
  - dst_host_count
  - dst_host_srv_diff_host_rate
  - level
  - service_ftp_data
  - service_other
  - service_vmnet
  - service_whois
  - flag_OTH
  - flag_REJ
  - flag_RSTO
  - flag_RSTOS0
  - flag_RSTR
  - flag_S0
  - flag_S1
  - flag_S2
  - flag_S3
  - flag_SF
  - flag_SH


In [29]:
print(X_rfeU2R.shape)

(67394, 20)


In [30]:
# Train decision tree classifier for U2R
clf_U2R = DecisionTreeClassifier(random_state=0)

# Fit classifier with class weights for extreme imbalance
from sklearn.utils.class_weight import compute_class_weight
u2r_class_weights = compute_class_weight('balanced', classes=np.unique(Y_U2R_binary), y=Y_U2R_binary)
u2r_class_weight_dict = dict(zip(np.unique(Y_U2R_binary), u2r_class_weights))
print(f"U2R Class weights: {u2r_class_weight_dict}")

clf_U2R = DecisionTreeClassifier(random_state=0, class_weight='balanced')
clf_U2R.fit(X_U2R, Y_U2R_binary)

U2R Class weights: {np.int64(0): np.float64(0.500386088919248), np.int64(1): np.float64(648.0192307692307)}


DecisionTreeClassifier(class_weight='balanced', random_state=0)

In [31]:
# Train classifier with RFE-selected features for U2R
clf_rfeU2R = DecisionTreeClassifier(random_state=0, class_weight='balanced')
clf_rfeU2R.fit(X_rfeU2R, Y_U2R_binary)

DecisionTreeClassifier(class_weight='balanced', random_state=0)

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

print("Evaluation Methodology Comparison:")
print("="*50)

# Cross-validation on training data (demonstrates inflated scores)
print("Cross-Validation on Training Data:")
accuracy = cross_val_score(clf_U2R, X_U2R, Y_U2R_binary, cv=10, scoring='accuracy')
print("   CV Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))

print("\nTrain-Test Evaluation:")
# Proper evaluation on test set
clf_U2R.fit(X_U2R, Y_U2R_binary)
Y_pred = clf_U2R.predict(X_U2R_test)

print("   Test Accuracy:", metrics.accuracy_score(Y_U2R_test_binary, Y_pred))
print("   Test Precision:", metrics.precision_score(Y_U2R_test_binary, Y_pred))
print("   Test Recall:", metrics.recall_score(Y_U2R_test_binary, Y_pred))
print("   Test F1:", metrics.f1_score(Y_U2R_test_binary, Y_pred))


Evaluation Methodology Comparison:
Cross-Validation on Training Data:
   CV Accuracy: 0.99967 (+/- 0.00046)

Train-Test Evaluation:
   Test Accuracy: 0.9871139292288812
   Test Precision: 0.2814814814814815
   Test Recall: 0.5671641791044776
   Test F1: 0.37623762376237624


In [33]:
# Apply RFE feature selection to U2R test data
X_U2R_test_rfe = X_U2R_test[:, rfecolindex_U2R]

# Make predictions using RFE-trained U2R classifier
Y_U2R_pred_rfe = clf_rfeU2R.predict(X_U2R_test_rfe)

# Confusion matrix for U2R classification
print(pd.crosstab(Y_U2R_test_binary, Y_U2R_pred_rfe, rownames=['Actual attacks'], colnames=['Predicted attacks']))

# Final evaluation on U2R test set
print("U2R Test Accuracy:", metrics.accuracy_score(Y_U2R_test_binary, Y_U2R_pred_rfe))
print("U2R Test Precision:", metrics.precision_score(Y_U2R_test_binary, Y_U2R_pred_rfe, zero_division=0))
print("U2R Test Recall:", metrics.recall_score(Y_U2R_test_binary, Y_U2R_pred_rfe))
print("U2R Test F1:", metrics.f1_score(Y_U2R_test_binary, Y_U2R_pred_rfe, zero_division=0))


Predicted attacks     0   1
Actual attacks             
0                  9710   1
1                    20  47
U2R Test Accuracy: 0.9978523215381468
U2R Test Precision: 0.9791666666666666
U2R Test Recall: 0.7014925373134329
U2R Test F1: 0.8173913043478261


In [34]:
# RFE Feature Selection Evaluation for U2R
print("RFE-based Feature Selection Performance for U2R:")
print("="*50)

# Train model with RFE-selected features for U2R
clf_rfeU2R.fit(X_rfeU2R, Y_U2R_binary)
Y_pred_rfe_U2R = clf_rfeU2R.predict(X_U2R_test[:, rfecolindex_U2R])

print("Test Performance with RFE Features for U2R:")
print(f"Test Accuracy: {metrics.accuracy_score(Y_U2R_test_binary, Y_pred_rfe_U2R):.5f}")
print(f"Test Precision: {metrics.precision_score(Y_U2R_test_binary, Y_pred_rfe_U2R, zero_division=0):.5f}")
print(f"Test Recall: {metrics.recall_score(Y_U2R_test_binary, Y_pred_rfe_U2R):.5f}")
print(f"Test F1-Score: {metrics.f1_score(Y_U2R_test_binary, Y_pred_rfe_U2R, zero_division=0):.5f}")


RFE-based Feature Selection Performance for U2R:
Test Performance with RFE Features for U2R:
Test Accuracy: 0.99785
Test Precision: 0.97917
Test Recall: 0.70149
Test F1-Score: 0.81739


In [35]:
X_U2R = np.array(X_U2R)
X_U2R_test = np.array(X_U2R_test)
# Keep data in 2D format for SMOTE - reshaping will be done after SMOTE application

In [36]:
Y_U2R_binary = np.array(Y_U2R_binary)
Y_U2R_test_binary = np.array(Y_U2R_test_binary)

In [37]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization, Attention, MultiHeadAttention
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek


In [38]:
# Advanced Class-Sensitive Focal Loss for Extreme Imbalance (U2R)
import tensorflow as tf
from tensorflow.keras import backend as K

def extreme_focal_loss(gamma=5.0, alpha=0.99, label_smoothing=0.1):
    """
    Enhanced Focal Loss for extreme class imbalance with label smoothing

    Args:
        gamma (float): High focusing parameter for extreme imbalance (5.0 vs 2.0)
        alpha (float): High weighting for minority class (0.99 for U2R)
        label_smoothing (float): Label smoothing to prevent overconfidence
    """
    def focal_loss_fixed(y_true, y_pred):
        # Apply label smoothing to prevent overconfidence
        y_true_smooth = y_true * (1 - label_smoothing) + 0.5 * label_smoothing

        # Ensure y_pred is in valid range [epsilon, 1-epsilon]
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate cross entropy
        ce_loss = -y_true_smooth * K.log(y_pred) - (1 - y_true_smooth) * K.log(1 - y_pred)

        # Calculate p_t (prediction probability for true class)
        p_t = y_true_smooth * y_pred + (1 - y_true_smooth) * (1 - y_pred)

        # Calculate alpha_t (class-sensitive weighting)
        alpha_t = y_true_smooth * alpha + (1 - y_true_smooth) * (1 - alpha)

        # Calculate focal loss with extreme focusing
        focal_weight = alpha_t * K.pow(1 - p_t, gamma)
        focal_loss = focal_weight * ce_loss

        return K.mean(focal_loss)

    return focal_loss_fixed

def class_sensitive_loss(gamma=5.0, alpha=0.99, penalty_factor=10.0):
    """
    Class-sensitive loss with penalty for missing U2R attacks
    """
    def loss_fn(y_true, y_pred):
        # Base focal loss
        focal = extreme_focal_loss(gamma, alpha)(y_true, y_pred)

        # Additional penalty for false negatives (missed U2R attacks)
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # False negative penalty: when y_true=1 but y_pred is low
        fn_penalty = penalty_factor * y_true * K.maximum(0.0, 0.5 - y_pred)

        return focal + K.mean(fn_penalty)

    return loss_fn

# Calculate extreme class weights for U2R
print(f"U2R Class distribution analysis:")
print(f"  Normal (0): {np.sum(Y_U2R_binary == 0):,} samples ({np.sum(Y_U2R_binary == 0)/len(Y_U2R_binary)*100:.4f}%)")
print(f"  U2R (1): {np.sum(Y_U2R_binary == 1):,} samples ({np.sum(Y_U2R_binary == 1)/len(Y_U2R_binary)*100:.4f}%)")

# Extreme class weights calculation
u2r_class_weights = compute_class_weight('balanced', classes=np.unique(Y_U2R_binary), y=Y_U2R_binary)
imbalance_ratio = np.sum(Y_U2R_binary == 0) / max(np.sum(Y_U2R_binary == 1), 1)

print(f"Imbalance ratio: {imbalance_ratio:.1f}:1")
print(f"Base class weights: {dict(zip(np.unique(Y_U2R_binary), u2r_class_weights))}")

# Enhanced weights - 10x penalty for missing U2R attacks
cost_sensitive_weights = {
    0: u2r_class_weights[0],
    1: u2r_class_weights[1] * 10.0  # 10x penalty for missing U2R
}

print(f"Cost-sensitive weights: {cost_sensitive_weights}")

# Set extreme focal loss parameters for U2R
alpha_focal = 0.99  # Very high weight for minority class
gamma_focal = 5.0   # High focusing parameter for extreme imbalance

print(f"Extreme Focal Loss parameters:")
print(f"  Alpha: {alpha_focal} (extreme minority weighting)")
print(f"  Gamma: {gamma_focal} (extreme focusing)")

# Create class-sensitive focal loss
focal_loss_fn = class_sensitive_loss(gamma=gamma_focal, alpha=alpha_focal, penalty_factor=10.0)

U2R Class distribution analysis:
  Normal (0): 67,342 samples (99.9228%)
  U2R (1): 52 samples (0.0772%)
Imbalance ratio: 1295.0:1
Base class weights: {np.int64(0): np.float64(0.500386088919248), np.int64(1): np.float64(648.0192307692307)}
Cost-sensitive weights: {0: np.float64(0.500386088919248), 1: np.float64(6480.192307692307)}
Extreme Focal Loss parameters:
  Alpha: 0.99 (extreme minority weighting)
  Gamma: 5.0 (extreme focusing)


In [39]:
import os

# Define a learning rate schedule with warmup for extreme imbalance
initial_learning_rate = 0.0001  # Lower LR for stability
def lr_schedule_u2r(epoch):
    """Learning rate schedule with warmup for extreme imbalance"""
    if epoch < 5:  # Warmup period
        return initial_learning_rate * (epoch + 1) / 5
    else:
        return initial_learning_rate * (0.95 ** ((epoch - 5) // 5))

output_dir = 'Models'
os.makedirs(output_dir, exist_ok=True)
model_path = os.path.join(output_dir, 'best_cnn_lstm_u2r_model.h5')

print(f"Model will be saved to: {model_path}")

my_callbacks = [
    ModelCheckpoint(
        filepath=model_path,
        verbose=1,
        save_best_only=True,
        monitor='val_recall',  # Focus on recall for U2R detection
        mode='max'
    ),
    EarlyStopping(
        monitor='val_recall',  # Monitor recall instead of loss
        patience=25,  # Increased patience for extreme imbalance
        restore_best_weights=True,
        verbose=1,
        min_delta=0.001,
        mode='max'
    ),
    LearningRateScheduler(lr_schedule_u2r, verbose=1)
]

Model will be saved to: Models/best_cnn_lstm_u2r_model.h5


In [44]:
# Advanced CNN-LSTM with Attention for Extreme Imbalance U2R Detection
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras import Model, Input

def create_lightweight_u2r_model(input_shape, num_classes=1):
    """
    Create lightweight CNN-LSTM model optimized for real-time U2R detection

    Real-time optimizations:
    1. Reduced layers and parameters (10x smaller)
    2. Simplified CNN with fewer filters
    3. Single LSTM layer
    4. Minimal dense layers
    5. Strategic dropout for performance balance
    """

    inputs = Input(shape=input_shape)

    # Lightweight CNN feature extraction (32->64 filters only)
    x = Conv1D(32, 3, padding="same", activation='relu')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x)

    x = Conv1D(64, 3, padding="same", activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)

    # Single efficient LSTM layer
    x = LSTM(32, dropout=0.2, recurrent_dropout=0.2)(x)

    # Minimal classification head (2 layers only)
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.4)(x)

    # Output layer
    outputs = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs, name="Lightweight_U2R_CNN_LSTM")
    return model

def create_ultra_light_u2r_model(input_shape, num_classes=1):
    """
    Create ultra-lightweight model for real-time deployment
    - Even faster inference
    - Minimal memory footprint
    - Optimized for edge deployment
    """

    inputs = Input(shape=input_shape)

    # Minimal CNN (single layer)
    x = Conv1D(16, 5, padding="same", activation='relu')(inputs)
    x = MaxPooling1D(pool_size=4)(x)
    x = Dropout(0.3)(x)

    # Small LSTM
    x = LSTM(16, dropout=0.2)(x)

    # Direct output
    outputs = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs, name="UltraLight_U2R_CNN_LSTM")
    return model

# Create lightweight models comparison
input_shape = (X_U2R.shape[1], 1)

print("Creating Real-Time U2R Detection Models...")
print("="*60)

# Option 1: Lightweight model (balanced performance/speed)
lightweight_model = create_lightweight_u2r_model(input_shape)
lightweight_params = lightweight_model.count_params()

# Option 2: Ultra-light model (maximum speed)
ultra_light_model = create_ultra_light_u2r_model(input_shape)
ultra_light_params = ultra_light_model.count_params()


# Choose the lightweight model for balanced performance
model = lightweight_model
model_params = lightweight_params

model.summary()

# Model compilation optimized for lightweight deployment
from tensorflow.keras.optimizers import Adam

# Use standard Adam optimizer for lightweight models (no weight decay overhead)
opt = Adam(
    learning_rate=initial_learning_rate,
    beta_1=0.9,
    beta_2=0.999
)

# Compile with essential metrics only (faster evaluation)
model.compile(
    loss=focal_loss_fn,
    optimizer=opt,
    metrics=[
        'accuracy',
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.Precision(name='precision')
    ]  # Reduced metrics for faster training/inference
)


# Advanced Training Strategy for Extreme Imbalance U2R Detection
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN

print(f"Original distribution - Normal: {np.sum(Y_U2R_binary == 0):,}, U2R: {np.sum(Y_U2R_binary == 1):,}")

# Stratified split with special handling for extreme imbalance
X_train, X_val, y_train, y_val = train_test_split(
    X_U2R, Y_U2R_binary,
    test_size=0.2,
    stratify=Y_U2R_binary,
    random_state=42
)

print(f"Split - Train: Normal {np.sum(y_train == 0):,}, U2R {np.sum(y_train == 1):,}")
print(f"Split - Val: Normal {np.sum(y_val == 0):,}, U2R {np.sum(y_val == 1):,}")

# Advanced SMOTE Ensemble for extreme imbalance
def create_advanced_synthetic_samples(X, y):
    """
    Create synthetic samples using multiple SMOTE variants for extreme imbalance
    """
    print("Applying Advanced SMOTE Ensemble...")

    # 1. ADASYN - Adaptive synthetic sampling
    adasyn = ADASYN(
        sampling_strategy=0.8,
        random_state=42,
        n_neighbors=3  # Reduced neighbors due to small minority class
    )

    try:
        X_adasyn, y_adasyn = adasyn.fit_resample(X, y)
        print(f"ADASYN result: Normal {np.sum(y_adasyn == 0):,}, U2R {np.sum(y_adasyn == 1):,}")
    except ValueError as e:
        print(f"ADASYN failed: {e}, falling back to BorderlineSMOTE")
        borderline = BorderlineSMOTE(
            sampling_strategy=0.3,
            random_state=42,
            k_neighbors=3,
            m_neighbors=3
        )
        X_adasyn, y_adasyn = borderline.fit_resample(X, y)
        print(f"BorderlineSMOTE result: Normal {np.sum(y_adasyn == 0):,}, U2R {np.sum(y_adasyn == 1):,}")

    # 2. SMOTE + Data Augmentation with noise
    X_augmented = X_adasyn.copy()
    y_augmented = y_adasyn.copy()

    # Add small random noise to synthetic samples (data augmentation)
    minority_idx = np.where(y_augmented == 1)[0]
    noise_factor = 0.01 * X_augmented.std(axis=0)

    for idx in minority_idx[len(np.where(y == 1)[0]):]:  # Only to synthetic samples
        noise = np.random.normal(0, noise_factor, X_augmented.shape[1])
        X_augmented[idx] += noise

    print(f"Final augmented: Normal {np.sum(y_augmented == 0):,}, U2R {np.sum(y_augmented == 1):,}")
    print(f"Imbalance ratio improved: {np.sum(y_augmented == 0) / np.sum(y_augmented == 1):.1f}:1")

    return X_augmented, y_augmented

# Apply advanced synthetic sampling
X_train_enhanced, y_train_enhanced = create_advanced_synthetic_samples(X_train, y_train)
X_val,y_val = create_advanced_synthetic_samples(X_val, y_val)

# Reshape for CNN input after synthetic sample generation
X_train_enhanced_cnn = np.reshape(X_train_enhanced, (X_train_enhanced.shape[0], X_train_enhanced.shape[1], 1))
X_val_cnn = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
X_test_cnn = np.reshape(X_U2R_test, (X_U2R_test.shape[0], X_U2R_test.shape[1], 1))

print(f"CNN input shapes:")
print(f"  Training: {X_train_enhanced_cnn.shape}")
print(f"  Validation: {X_val_cnn.shape}")
print(f"  Test: {X_test_cnn.shape}")

# Enhanced callbacks with focus on speed and recall
my_callbacks_u2r = [
    ModelCheckpoint(
        filepath=model_path,
        verbose=1,
        save_best_only=True,
        monitor='val_accuracy',  # Critical for U2R detection
        mode='max'
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=20,  # Reduced patience for faster training
        restore_best_weights=True,
        verbose=1,
        min_delta=0.001,
        mode='max'
    ),
    LearningRateScheduler(lr_schedule_u2r, verbose=1)
]


history = model.fit(
    X_train_enhanced_cnn, y_train_enhanced,
    epochs=60,  # Reduced epochs for lightweight model
    batch_size=64,  # Larger batch size for efficiency
    validation_data=(X_val_cnn, y_val),
    class_weight=cost_sensitive_weights,
    callbacks=my_callbacks_u2r,
    verbose=1
)



Creating Real-Time U2R Detection Models...


Model: "Lightweight_U2R_CNN_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 123, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_12 (Conv1D)              │ (None, 123, 32)        │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_12 (MaxPooling1D) │ (None, 61, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 61, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 61, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_13 (MaxPooling1D) │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,297 (75.38 KB)

 Trainable params: 19,297 (75.38 KB)

...

1516/1516 ━━━━━━━━━━━━━━━━━━━━ 82s 49ms/step - accuracy: 0.9927 - loss: 0.6957 - precision: 0.9838 - recall: 0.9999 - val_accuracy: 0.9980 - val_loss: 4.0151e-05 - val_precision: 0.9955 - val_recall: 1.0000 - learning_rate: 7.3509e-05

Epoch 37: LearningRateScheduler setting learning rate to 7.350918906249998e-05.
Epoch 37/60
1515/1516 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9921 - loss: 0.5077 - precision: 0.9826 - recall: 0.9999
Epoch 37: val_accuracy improved from 0.99798 to 0.99802, saving model to Models/best_cnn_lstm_u2r_model.h5


1516/1516 ━━━━━━━━━━━━━━━━━━━━ 82s 49ms/step - accuracy: 0.9921 - loss: 0.5075 - precision: 0.9826 - recall: 0.9999 - val_accuracy: 0.9980 - val_loss: 3.8312e-05 - val_precision: 0.9956 - val_recall: 1.0000 - learning_rate: 7.3509e-05

Epoch 38: LearningRateScheduler setting learning rate to 7.350918906249998e-05.
Epoch 38/60
1515/1516 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9936 - loss: 0.3409 - precision: 0.9858 - recall: 1.0000
Epoch 38: val_accuracy improved from 0.99802 to 0.99806, saving model to Models/best_cnn_lstm_u2r_model.h5


1516/1516 ━━━━━━━━━━━━━━━━━━━━ 83s 50ms/step - accuracy: 0.9936 - loss: 0.3409 - precision: 0.9858 - recall: 1.0000 - val_accuracy: 0.9981 - val_loss: 4.2118e-05 - val_precision: 0.9957 - val_recall: 1.0000 - learning_rate: 7.3509e-05

Epoch 39: LearningRateScheduler setting learning rate to 7.350918906249998e-05.
Epoch 39/60
1516/1516 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9945 - loss: 0.3306 - precision: 0.9879 - recall: 1.0000
Epoch 39: val_accuracy did not improve from 0.99806
1516/1516 ━━━━━━━━━━━━━━━━━━━━ 80s 49ms/step - accuracy: 0.9945 - loss: 0.3306 - precision: 0.9879 - recall: 1.0000 - val_accuracy: 0.9981 - val_loss: 3.7452e-05 - val_precision: 0.9957 - val_recall: 1.0000 - learning_rate: 7.3509e-05

Epoch 40: LearningRateScheduler setting learning rate to 7.350918906249998e-05.
Epoch 40/60
1515/1516 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9941 - loss: 0.3753 - precision: 0.9870 - recall: 1.0000
Epoch 40: val_accuracy improved from 0.99806 to 0.99823, savin

1516/1516 ━━━━━━━━━━━━━━━━━━━━ 76s 50ms/step - accuracy: 0.9941 - loss: 0.3753 - precision: 0.9870 - recall: 1.0000 - val_accuracy: 0.9982 - val_loss: 3.8543e-05 - val_precision: 0.9960 - val_recall: 1.0000 - learning_rate: 7.3509e-05

Epoch 41: LearningRateScheduler setting learning rate to 6.983372960937497e-05.
Epoch 41/60
1515/1516 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9935 - loss: 0.3471 - precision: 0.9857 - recall: 0.9999
Epoch 41: val_accuracy did not improve from 0.99823
1516/1516 ━━━━━━━━━━━━━━━━━━━━ 77s 51ms/step - accuracy: 0.9935 - loss: 0.3470 - precision: 0.9857 - recall: 0.9999 - val_accuracy: 0.9981 - val_loss: 3.7227e-05 - val_precision: 0.9958 - val_recall: 1.0000 - learning_rate: 6.9834e-05

Epoch 42: LearningRateScheduler setting learning rate to 6.983372960937497e-05.
Epoch 42/60
1515/1516 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9910 - loss: 0.8117 - precision: 0.9803 - recall: 0.9999
Epoch 42: val_accuracy did not improve from 0.99823
1516/1516 

1516/1516 ━━━━━━━━━━━━━━━━━━━━ 76s 50ms/step - accuracy: 0.9947 - loss: 0.2608 - precision: 0.9883 - recall: 1.0000 - val_accuracy: 0.9983 - val_loss: 3.5027e-05 - val_precision: 0.9962 - val_recall: 1.0000 - learning_rate: 6.6342e-05

Epoch 49: LearningRateScheduler setting learning rate to 6.634204312890623e-05.
Epoch 49/60
1516/1516 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9930 - loss: 0.3480 - precision: 0.9845 - recall: 1.0000
Epoch 49: val_accuracy did not improve from 0.99831
1516/1516 ━━━━━━━━━━━━━━━━━━━━ 81s 49ms/step - accuracy: 0.9930 - loss: 0.3480 - precision: 0.9845 - recall: 1.0000 - val_accuracy: 0.9981 - val_loss: 3.5606e-05 - val_precision: 0.9957 - val_recall: 1.0000 - learning_rate: 6.6342e-05
Epoch 49: early stopping
Restoring model weights from the end of the best epoch: 29.


In [45]:
# Advanced U2R Model Evaluation with Focus on Recall
print("Evaluating Advanced U2R Detection Model...")

# Predictions
y_pred_proba = model.predict(X_test_cnn, verbose=0)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print(f"Prediction shape: {y_pred.shape}")
print(f"Test labels shape: {Y_U2R_test_binary.shape}")

Evaluating Advanced U2R Detection Model...
Prediction shape: (9778,)
Test labels shape: (9778,)


In [46]:
# Threshold optimization for U2R detection
from sklearn.metrics import precision_recall_curve, roc_curve, auc

def find_optimal_threshold_u2r(y_true, y_proba):
    """
    Find optimal threshold that balances precision and recall for U2R detection
    Prioritizes recall due to critical nature of U2R attacks
    """
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_proba)

    # F2-score prioritizes recall (beta=2)
    f2_scores = (5 * precisions * recalls) / (4 * precisions + recalls + 1e-8)
    optimal_idx = np.argmax(f2_scores)
    optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5

    print(f"Optimal threshold analysis:")
    print(f"  Threshold: {optimal_threshold:.3f}")
    print(f"  Precision: {precisions[optimal_idx]:.3f}")
    print(f"  Recall: {recalls[optimal_idx]:.3f}")
    print(f"  F2-Score: {f2_scores[optimal_idx]:.3f}")

    return optimal_threshold

# Find optimal threshold
optimal_threshold = find_optimal_threshold_u2r(Y_U2R_test_binary, y_pred_proba.flatten())

# Apply optimal threshold
y_pred_optimal = (y_pred_proba > optimal_threshold).astype(int).flatten()

Optimal threshold analysis:
  Threshold: 0.942
  Precision: 0.505
  Recall: 0.731
  F2-Score: 0.671


In [47]:
# Final predictions with optimal threshold
y_pred_final = y_pred_optimal
print(f"Final predictions using optimal threshold: {optimal_threshold:.3f}")
print(f"Predicted U2R attacks: {np.sum(y_pred_final == 1)}")
print(f"Actual U2R attacks: {np.sum(Y_U2R_test_binary == 1)}")

Final predictions using optimal threshold: 0.942
Predicted U2R attacks: 96
Actual U2R attacks: 67


In [51]:
# Comprehensive U2R Detection Evaluation
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

print("="*80)
print("ADVANCED U2R DETECTION RESULTS")
print("="*80)

# Detailed metrics calculation
accuracy = accuracy_score(Y_U2R_test_binary, y_pred_final)
balanced_acc = balanced_accuracy_score(Y_U2R_test_binary, y_pred_final)
precision = precision_score(Y_U2R_test_binary, y_pred_final, zero_division=0)
recall = recall_score(Y_U2R_test_binary, y_pred_final)
f1 = f1_score(Y_U2R_test_binary, y_pred_final, zero_division=0)
f2 = (5 * precision * recall) / (4 * precision + recall + 1e-8)

# ROC-AUC
try:
    roc_auc = roc_auc_score(Y_U2R_test_binary, y_pred_proba.flatten())
    print(f"ROC-AUC Score: {roc_auc:.4f}")
except:
    print("ROC-AUC: Could not calculate")



# Confusion Matrix Analysis
cm = confusion_matrix(Y_U2R_test_binary, y_pred_final)
print(f"\nConfusion Matrix:")
print(cm)

# Detailed confusion matrix breakdown
tn, fp, fn, tp = cm.ravel()

# Attack detection analysis
total_u2r = np.sum(Y_U2R_test_binary == 1)
if total_u2r > 0:
    detection_rate = tp / total_u2r
    miss_rate = fn / total_u2r
    print(f"\nU2R Attack Detection Analysis:")
    print(f"  Total U2R attacks: {total_u2r}")
    print(f"  Successfully detected: {tp} ({detection_rate*100:.1f}%)")
    print(f"  Missed attacks: {fn} ({miss_rate*100:.1f}%)")

    if fn > 0:
        print(f"  CRITICAL: {fn} U2R attacks went undetected!")
else:
    print("No U2R attacks in test set")

# Cost analysis
false_alarm_rate = fp / (tn + fp) if (tn + fp) > 0 else 0
print(f"\nOperational Impact:")
print(f"  False Alarm Rate: {false_alarm_rate*100:.2f}%")
print(f"  Security Risk: {fn} critical U2R attacks undetected")

print("\nClassification Report:")
print(classification_report(Y_U2R_test_binary, y_pred_final,
                          target_names=['Normal', 'U2R Attack'],
                          digits=4))



ADVANCED U2R DETECTION RESULTS
ROC-AUC Score: 0.9682

Confusion Matrix:
[[9663   48]
 [  19   48]]

U2R Attack Detection Analysis:
  Total U2R attacks: 67
  Successfully detected: 48 (71.6%)
  Missed attacks: 19 (28.4%)
  CRITICAL: 19 U2R attacks went undetected!

Operational Impact:
  False Alarm Rate: 0.49%
  Security Risk: 19 critical U2R attacks undetected

Classification Report:
              precision    recall  f1-score   support

      Normal     0.9980    0.9951    0.9965      9711
  U2R Attack     0.5000    0.7164    0.5890        67

    accuracy                         0.9931      9778
   macro avg     0.7490    0.8557    0.7928      9778
weighted avg     0.9946    0.9931    0.9938      9778

